**Using this template**

1. Change notebook title to assignment title you working on, eg. `Graded Challenge 1 or Milestone 1`
2. Put your identity such as name and batch number below notebook title.
3. Write your description before do any work. State your purpose of this assignment and what you are trying to achieve. 
4. Look at the rubric of this assignment and make sure you understand the criteria.
5. Code your solution in the cell provided below the working area section.
6. State your conlusions, findings, and any other relevant information in the cell provided below the conclusions sections.
7. Save this notebook and rename it to assignment title you working on  eg. `Graded Challenge 1 or Milestone 1`
8. Push your assignment before deadline.
9. Good Luck!

# Graded Challenge 2

1. Name: Jason Rich Darmawan Onggo Putra
2. Batch: 016 RMT

## Description

### Objective

Analysis of the waste management and disposal system in the city of Austin. The analysis result will be the condition of the waste management and disposal system. In addition, the prediction of the weight of waste in the city of Austin on July 10, 2021.

### Problem Statement

#### Problems 1

- [ ] What is the average **time between delivery** of every dropoff site? 
= [ ] What is the **maximum** waste weight collected of every dropoff site **per day**?
- [ ] What is the **average** waste weight collected of every dropoff site **per delivery**?
- [ ] What is the **median** waste weight collected of every dropoff site **per delivery**?
- [ ] What is the **mode** waste weight of every dropoff site **per delivery**?

### Problems 2

- [ ] What is the **maximum** waste weight collected of every dropoff site in July 10, 2021?

## Working Area

```txt
Put your code, analysis, everything below this line

Make sure to check everything the rubric requires before you submit
```

## Data Retrieve

1. project: `bigquery-public-data`
2. dataset: [austin_waste](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=samples&page=dataset&_ga=2.245085957.1471931019.1642739417-486643658.1638156099&project=lexical-period-361812&ws=!1m4!1m3!3m2!1sbigquery-public-data!2saustin_waste)
3. table: [waste_and_diversion](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=samples&page=dataset&_ga=2.245085957.1471931019.1642739417-486643658.1638156099&project=lexical-period-361812&ws=!1m5!1m4!4m3!1sbigquery-public-data!2saustin_waste!3swaste_and_diversion)

**The query**:
```
SELECT report_date, load_time, load_type, load_weight, dropoff_site 
FROM `bigquery-public-data.austin_waste.waste_and_diversion`
WHERE report_date BETWEEN '2021-01-01' AND '2021-07-09';
```

## Import Library

In [2]:
import pandas as pd

## Data Loading

In [114]:
df = pd.read_csv("./h8dsft_P0GC2_jason_rich_darmawan_onggo_putra.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26157 entries, 0 to 26156
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   report_date   26157 non-null  object 
 1   load_time     26157 non-null  object 
 2   load_type     26157 non-null  object 
 3   load_weight   25109 non-null  float64
 4   dropoff_site  26157 non-null  object 
dtypes: float64(1), object(4)
memory usage: 1021.9+ KB


In [115]:
df.head()

,report_date,load_time,load_type,load_weight,dropoff_site
0,2021-07-08,2021-07-08 12:00:00 UTC,TIRES,"2,260.00",MRF
1,2021-07-08,2021-07-08 11:00:00 UTC,TIRES,"1,260.00",MRF
2,2021-07-08,2021-07-08 03:00:00 UTC,TIRES,800.00,MRF
3,2021-05-01,2021-05-01 12:29:00 UTC,TIRES,760.00,MRF
4,2021-07-03,2021-07-03 12:09:00 UTC,TIRES,"1,400.00",MRF


In [6]:
df.tail()

,report_date,load_time,load_type,load_weight,dropoff_site
26152,2021-03-01,2021-03-01 11:26:00 UTC,ORGANICS,12100.0,ORGANICS BY GOSH
26153,2021-03-01,2021-03-01 04:27:00 UTC,ORGANICS,10900.0,ORGANICS BY GOSH
26154,2021-03-01,2021-03-01 05:55:00 UTC,ORGANICS,21800.0,ORGANICS BY GOSH
26155,2021-03-01,2021-03-02 01:10:00 UTC,ORGANICS,5640.0,ORGANICS BY GOSH
26156,2021-03-01,2021-03-01 10:22:00 UTC,RECYCLED METAL,360.0,AUSTIN IRON AND METAL


## Data Cleaning

## Data Exploration

## Conclusions, Assumptions, Overall Analysis

`put your conclusions here`

## Playground

### Problem 1

2. eksplorasi data
a. rentang waktu pengambilan data
b. tipe load sampah
```
df.loc[:,'load_type'].unique()
```
c. tempat pembuangan sampah
```
df.loc[:,'dropoff_site'].unique()
```

3. insight/infromasi untuk masing-masing site.
```
df.loc[df['dropoff_site'] == 'MRF'].groupby(['dropoff_site'])['load_weight'].agg([pd.Series.mean, pd.Series.median, pd.Series.mode])
```

4. site paling menarik + alasan

5. site tersebut: column `load_weight` ada outlier? berapa persen? bandingkan mean, median, mode setelah datanya dibersihkan.

6. site tersebut: range, variance, standar deviasi column `load_weight`

pg_df_MRF = df.loc[df['dropoff_site'] == 'MRF','load_weight']
pg_df_MRF.max() - pg_df_MRF.min()
pg_df_MRF.var()
pg_df_MRF.std() # np.sqrt(pandas.Series.var())

7. menambah site baru: berapa kapasitas penampungan di site baru berdasarkan perhitungan confidence interval.

8. uji hipotesis

1. teknik handling outlier

2. konsep dibaling confidence interval

3. uji hipotesis

### Problem 2

forecasting.

v = Δw / Δt 

In [116]:
df.loc[:,'load_type'].unique()
df.loc[:,'dropoff_site'].unique()
# force pandas to suppress the scientific notation
pd.options.display.float_format = '{:,.2f}'.format
df.loc[df['dropoff_site'] == 'MRF'].groupby(['dropoff_site'])['load_weight'].agg([pd.Series.mean, pd.Series.median, pd.Series.mode])

,mean,median,mode
dropoff_site,,,
MRF,"1,799.98","1,360.00",960.00
